# Electricity uptime

In the notebook ```2016-02-10-electricity-uptime.ipynb``` I showed a method to estimate the uptime of the grid.  This notebook applies this method to the grids for which we have some data.  This method is very naive and does not include checks with the communication logs, or consistency with the delivered amounts of electricity.

The grids show uptimes of 85% or better while the microgrids are from 14% to 22% uptime.  I'd like to know how close to the desired schedule this is.

## Effect of Intermittency on Uptime

- Given that the microgrids have low uptime, how much does this constrain use?  
- Is the average daily uptime (in contiguous blocks) long enough to match with the survey questions?

In [1]:
import pandas as pd
import numpy as np
from pint import UnitRegistry

def parse_file(energy_file_name):
    # read in raw CSV file for preprocessing
    energy_file = open(energy_file_name)
    # append data lines after header beginning with 'Timestamp'
    # dump lines from communications messages or empty lines and write to string
    file_string = ''
    omit_strings = ['Power Up', 'Communication Lost', 'Power Down', ',,,,,,,,,,,']
    # append data not matching omit_strings after sentinel is reached
    data_found = False
    for line in energy_file.readlines():
        if 'Timestamp' in line:
            data_found = True
        if not any(omit in line for omit in omit_strings) and data_found:
            file_string += line
    return file_string    

def create_data_frame(energy_file_name):
    # use pre-parsed string to read in data file to pandas and sort by date
    file_string = parse_file(energy_file_name)
    import io
    energy_data = pd.read_csv(io.StringIO(file_string),
                          index_col=0, 
                          parse_dates=[0], 
                          thousands=',')
    energy_data = energy_data.sort_index()
    return energy_data

def get_energy_stats(energy_data):
    energy = energy_data['kWh export']
    resampled = energy.resample('D').diff()
    return resampled.mean(), resampled.median()

def calculate_uptime(energy_data):
    # take the discrete difference between time samples
    # convert to seconds from nanoseconds
    # put in pandas series
    differences = np.diff(energy_data.index.values)/1e9
    differences = differences.astype(int)
    diff_series = pd.Series(differences)

    # use value counts to see intervals
    dsvc = diff_series.value_counts()
    dsvc.sort_index()

    # intervals greater than 60 seconds are considered downtime
    cutoff_seconds = 60
    uptime = dsvc[dsvc.index <= cutoff_seconds]
    downtime = dsvc[dsvc.index > cutoff_seconds]
    
    # determine total downtime and uptime
    u = UnitRegistry()
    inferred_uptime = (uptime.index * uptime).values.sum() * u.second
    inferred_downtime = (downtime.index * downtime).values.sum() * u.second
    
    # use total observation period to get fraction of uptime
    clock_time = (energy_data.index[-1] - energy_data.index[0]).total_seconds() * u.seconds
    fraction_available = inferred_uptime / clock_time
    
    return fraction_available

In [4]:
energy_file_names = (('Ajau', 121, "../data-timeseries/Data_Logs Ajau Agustus.csv"),
                     ('Atamali', 40, "../data-timeseries/Data_Logs Atamali Agustus.csv"),
                     ('Ayapo', 103, "../data-timeseries/Data_Logs Ayapo Agustus.csv"),
                     ('Asei', 33, "../data-timeseries/Data_Logs ASEI July.csv"),
                     ('Kensio', 20, "../data-timeseries/Data_Logs Kensio Agustus.csv"))

output = {}
verbose = False
for village, connections, energy_file_name in energy_file_names: 
    energy_data = create_data_frame(energy_file_name)
    mean, median = get_energy_stats(energy_data)
    fraction_available = calculate_uptime(energy_data)
    if verbose: print(village, mean, median, fraction_available)
    output[village] = {'mean kWh per day': mean,
                       'mean kWh per day per household': mean/connections,
                       'median kWh per day': median,
                       'uptime' : fraction_available.magnitude}

output_data = pd.DataFrame(output).T
output_data

,mean kWh per day,mean kWh per day per household,median kWh per day,uptime
Ajau,611.081857,5.050263,623.048111,0.963965
Asei,186.858382,5.662375,213.208790,0.849619
Atamali,15.370199,0.384255,16.787386,0.200375
Ayapo,95.387185,0.926089,97.459428,0.226117
Kensio,10.781886,0.539094,9.852627,0.143118
